#### Restart and Run All

In [1]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
#data_path = "\\users\\user\\onedrive\\a7\\Data"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
today

datetime.date(2023, 4, 27)

### Today = Last business day

In [2]:
#today = date(2023, 3, 13)
today

datetime.date(2023, 4, 27)

In [3]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(3)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-04-27
yesterday: 2023-04-24


In [4]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

### Begin of Tables in the process

In [5]:
sql = """
SELECT * 
FROM sales 
ORDER BY id DESC LIMIT 1
"""
tmp = pd.read_sql(sql, conlite)
tmp["fm_date"] = pd.to_datetime(tmp["fm_date"])
tmp["to_date"] = pd.to_datetime(tmp["to_date"])
tmp["created_at"] = pd.to_datetime(tmp["created_at"])
tmp["updated_at"] = pd.to_datetime(tmp["updated_at"])
tmp.style.format(format_dict)

,id,name,fm_date,to_date,days,fm_price,to_price,diff,pct,ttl_spread,avg_spread,max_price,min_price,qty,created_at,updated_at,latest_date_id
0,1250727,SYNEX,2023-04-26,2023-04-26,0,12.80,12.90,0.10,0.78%,1,0,14.20,12.80,"5,929,024",2023-04-26,2023-04-26,1


In [6]:
names = tmp["name"]
type(names)

pandas.core.series.Series

In [7]:
name = names.to_string(index=False)
type(name)

str

In [8]:
sql = """
SELECT * 
FROM stocks
WHERE name = '%s'
"""
sql = sql % name
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.style.format(format_dict)


SELECT * 
FROM stocks
WHERE name = 'SYNEX'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,603,SYNEX,37.00,23.00,B,12.50,21.50,103.17,1.32,28.70,"3,000",-10,15,"15,000",1,12,RND55pct,SET


### End of Tables in the process

In [9]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2023-04-24'
ORDER BY a.name



,name
0,BCH
1,GVREIT
2,JASIF
3,JMART
4,JMT


In [10]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2023-04-24' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [11]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,JMART,2023-04-24,2023-04-24,19.10,19.40,"106,301,812",22.00,18.80,B,SET50,0.30,1.57%
1,JMT,2023-04-21,2023-04-24,39.25,40.50,"23,392,700",40.50,38.25,B,SET50,1.25,3.18%
2,BCH,2023-04-21,2023-04-24,21.70,21.50,"18,607,526",21.70,21.30,I,SET100,-0.20,-0.92%
3,GVREIT,2023-04-17,2023-04-24,9.30,9.00,"2,102,298",9.45,9.00,I,SET,-0.30,-3.23%
4,JASIF,2023-04-24,2023-04-24,7.00,7.05,"62,967,060",7.15,6.95,S,SET,0.05,0.71%


In [12]:
df.shape

(5, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [13]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,JMART,2023-04-24,2023-04-24,19.10,19.40,"106,301,812",22.00,18.80,1.57%,B
1,JMT,2023-04-21,2023-04-24,39.25,40.50,"23,392,700",40.50,38.25,3.18%,B
2,BCH,2023-04-21,2023-04-24,21.70,21.50,"18,607,526",21.70,21.30,-0.92%,I
3,GVREIT,2023-04-17,2023-04-24,9.30,9.00,"2,102,298",9.45,9.00,-3.23%,I
4,JASIF,2023-04-24,2023-04-24,7.00,7.05,"62,967,060",7.15,6.95,0.71%,S


In [14]:
file_name = "daily-sales.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(one_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [15]:
df[cols].shape

(5, 10)

In [16]:
sales = df[cols]
sales

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,JMART,2023-04-24,2023-04-24,19.10,19.40,106301812,22.00,18.80,1.57,B
1,JMT,2023-04-21,2023-04-24,39.25,40.50,23392700,40.50,38.25,3.18,B
2,BCH,2023-04-21,2023-04-24,21.70,21.50,18607526,21.70,21.30,-0.92,I
3,GVREIT,2023-04-17,2023-04-24,9.30,9.00,2102298,9.45,9.00,-3.23,I
4,JASIF,2023-04-24,2023-04-24,7.00,7.05,62967060,7.15,6.95,0.71,S


In [17]:
file_name = "Price-Today.csv"
input_file = data_path + file_name
prices = pd.read_csv(input_file)
prices

,name,date,price,change,%change,open,high,low,volume,value(m)
0,ACE,2023-04-27,1.99,0.01,0.505051,2.00,2.02,1.97,14639262,2.920112e+04
1,ADVANC,2023-04-27,214.00,2.00,0.943396,213.00,216.00,212.00,11134929,2.379005e+06
2,AEONTS,2023-04-27,190.00,2.00,1.063830,189.00,190.50,187.50,264563,5.003452e+04
3,AH,2023-04-27,31.75,-0.25,-0.781250,32.00,32.25,31.50,793010,2.520392e+04
4,AIE,2023-04-27,1.83,-0.03,-1.612903,1.85,1.87,1.82,185149,3.396002e+02
...,...,...,...,...,...,...,...,...,...,...
187,WHAIR,2023-04-27,7.45,0.00,0.000000,7.45,7.45,7.40,54502,4.043718e+02
188,WHART,2023-04-27,11.00,0.00,0.000000,10.90,11.00,10.90,313801,3.447561e+03
189,WHAUP,2023-04-27,3.78,-0.10,-2.577320,3.78,3.80,3.76,3512248,1.328914e+04
190,WICE,2023-04-27,8.15,-0.30,-3.550296,8.50,8.50,8.15,2162927,1.787900e+04


In [18]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,date,price,change,%change,open,high,low,volume,value(m)
0,JMART,2023-04-24,2023-04-24,19.10,19.40,106301812,22.00,18.80,1.57,B,2023-04-27,18.60,0.20,1.086957,18.30,18.70,17.80,15271833,277845.22410
1,JMT,2023-04-21,2023-04-24,39.25,40.50,23392700,40.50,38.25,3.18,B,2023-04-27,38.75,-0.50,-1.273885,39.00,39.25,37.75,9169121,352608.43125
2,BCH,2023-04-21,2023-04-24,21.70,21.50,18607526,21.70,21.30,-0.92,I,2023-04-27,21.90,0.20,0.921659,21.70,22.00,21.50,14519673,316449.69570
3,GVREIT,2023-04-17,2023-04-24,9.30,9.00,2102298,9.45,9.00,-3.23,I,2023-04-27,9.20,0.10,1.098901,9.10,9.20,9.10,98500,903.56000
4,JASIF,2023-04-24,2023-04-24,7.00,7.05,62967060,7.15,6.95,0.71,S,2023-04-27,7.05,-0.05,-0.704225,7.05,7.15,7.05,6607832,46900.09465


In [19]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
colu

['name',
 'fm_date',
 'to_date',
 'fm_price',
 'to_price',
 'qty',
 'max_price',
 'min_price',
 'percent',
 'status',
 'price',
 'change',
 'volume',
 'date']

In [20]:
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,JMART,2023-04-24,2023-04-24,19.10,19.40,106301812,22.00,18.80,1.57,B,18.60,0.20,15271833,2023-04-27
1,JMT,2023-04-21,2023-04-24,39.25,40.50,23392700,40.50,38.25,3.18,B,38.75,-0.50,9169121,2023-04-27
2,BCH,2023-04-21,2023-04-24,21.70,21.50,18607526,21.70,21.30,-0.92,I,21.90,0.20,14519673,2023-04-27
3,GVREIT,2023-04-17,2023-04-24,9.30,9.00,2102298,9.45,9.00,-3.23,I,9.20,0.10,98500,2023-04-27
4,JASIF,2023-04-24,2023-04-24,7.00,7.05,62967060,7.15,6.95,0.71,S,7.05,-0.05,6607832,2023-04-27


In [21]:
df_merge[colu].shape

(5, 14)

In [22]:
df_merge.shape

(5, 19)

In [23]:
file_name = "daily-sales-prices.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(data_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(one_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

In [24]:
### Call ruby ruby\daily-out-new.rb

In [25]:
%pwd

'C:\\Users\\User\\OneDrive\\A7\\Daily'

In [26]:
os.chdir("C:\\users\\user\\onedrive\\a7")
%pwd

'C:\\users\\user\\onedrive\\a7'

In [27]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
BCH      2023-04-27 2023-04-27  21.50  21.90   1.86%  18,607,526  21.90  21.30 I Insert
GVREIT   2023-04-27 2023-04-27   9.00   9.20   2.22%   2,102,298   9.45   9.00 I Insert
JASIF    2023-04-27 2023-04-27   7.05   7.05    0.0%  62,967,060   7.15   6.95 S Insert
JMART    2023-04-24 2023-04-27  19.10  18.60  -2.62% 106,301,812  22.00  18.60 B Update
JMT      2023-04-27 2023-04-27  40.50  38.75  -4.32%  23,392,700  40.50  38.25 B Insert


In [28]:
os.chdir("C:\\users\\user\\onedrive\\a7\\Daily")
%pwd

'C:\\users\\user\\onedrive\\a7\\Daily'

### cd\ruby\port_lite
### rails runner db/ins_sales.rb